In [ ]:
import pandas as pd
import utils.fetcher_utils as fetcher
import utils.pipeline_util as pu
import utils.transformer_util as tu

In [ ]:
# set pandas options
pd.set_option('display.max_columns', None)

In [ ]:
# fetch the data 
df = fetcher.aquireIMDbDataFrame()
df.head()

In [ ]:
# Run pipeline to get trained model to use for predictions
model, y_pred, processed_df = pu.run_pipeline( data=df, use_PCA=False, debug=False)

In [ ]:
# Use builder to create a new DataFrame to use with trained model from pipeline and predict the IMDB score

In [ ]:
# Showcase the prediction with Average, Median and Mode along with quartiles